# Generating strain growing on xylose

For this test case: 
- consumption needed
- production not needed
- optknock maybe. 

### Simulate the strain growing on D-xylose
- delete (G1Dx or GDH2) GLCDpp 
- Addition of XYLI1 xylose isomerase
- Xylulokinase XYlK
- D-xylose:H+ symporter XYLt2

### 1. making the modifications manually and use GF only for growth on arabinose

In [1]:
from os.path import join
import cobra
import csv
from cobra.flux_analysis import phenotype_phase_plane
from cobra.flux_analysis import gapfill

from pipeline.scripts.analysis import analysis_gf_sol, dict_prod_sol, cons_prod_dict
from pipeline.scripts.import_models import get_universal_main, get_reference_model
from pipeline.scripts.input_parser import main
from pipeline.scripts.scoring_system import (scores_evaluations, score_BB_presence, 
                            plotting_KI_BB_scores)
from pipeline.scripts.to_and_from_biobrick_wikidata import (output_for_biobrick_search, \
                                        run_harmonization_per_model)
from pipeline.scripts.call_Optknock import (run_optknock_analysis, \
                                        add_Optknock_to_analysis)
from pipeline.scripts.output_generation import (summary_output, metab_out_chain, 
                            write_data_provenance)

DEBUG:matplotlib:(private) matplotlib data path: C:\Users\delip\anaconda3\envs\cameo\lib\site-packages\matplotlib\mpl-data
DEBUG:matplotlib:matplotlib data path: C:\Users\delip\anaconda3\envs\cameo\lib\site-packages\matplotlib\mpl-data
DEBUG:matplotlib:CONFIGDIR=C:\Users\delip\.matplotlib
DEBUG:matplotlib:matplotlib version 3.3.0
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is win32
DEBUG:matplotlib:loaded modules: ['builtins', 'sys', '_frozen_importlib', '_imp', '_warnings', '_thread', '_weakref', '_frozen_importlib_external', '_io', 'marshal', 'nt', 'winreg', 'zipimport', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_weakrefset', '_bootlocale', '_locale', 'encodings.cp1252', 'site', 'os', 'errno', 'stat', '_stat', 'ntpath', 'genericpath', 'os.path', '_collections_abc', '_sitebuiltins', 'sysconfig', 'types', 'functools', '_functools', 'collections', 'operator', '_operator', 'keywo

DEBUG:matplotlib:CACHEDIR=C:\Users\delip\.matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from C:\Users\delip\.matplotlib\fontlist-v330.json
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


Using license file C:\Users\delip\gurobi.lic


INFO:gurobipy.gurobipy:Using license file C:\Users\delip\gurobi.lic


Academic license - for non-commercial use only


INFO:gurobipy.gurobipy:Academic license - for non-commercial use only


No parameters matching '_test' found


INFO:gurobipy.gurobipy:No parameters matching '_test' found


In [2]:
data_repo = "./pipeline/inputs/"
model = get_reference_model(data_repo, './pipeline/inputs/putida_ara_meth.csv')

In [3]:
data_dir = cobra.test.data_dir
mock = cobra.Model("ch4_reactions")
#mock = cobra.io.read_sbml_model(join(data_dir, 'mock_ch4_fald_all.xml'))

In [4]:
main('./pipeline/inputs/putida_ara_meth.csv', mock, model)


arab__L is not in the medium 

Ther reaction EX_arab__L_e has been added to the reference model, hence arab__L_e is now in the medium
Exchange arab__L:  arab__L_e <--  Old bounds:  (-1000, 0)

akg is in the medium 

Exchange akg:  akg_e -->  Old bounds:  (0.0, 999999.0)
For arab__L there isn't any uptake trasnsporter in the reference model
The trasporter has been added to the 
                                reference model from the input file
For akg there isn't any uptake trasnsporter in the reference model

The following reactions will be added to the universal model: ['ARAI', 'RBK_L1', 'RBP4E', 'ARABD', 'LARLC', 'LARNDH', 'L2K3dARNDH', 'XYLI1', 'XYLK', 'XYLt2', 'R01143', 'MMO1', 'MMO2', 'ALCD1']
unknown metabolite 'arab__L_c' created
unknown metabolite 'rbl__L_c' created
unknown metabolite 'atp_c' created
unknown metabolite 'adp_c' created
unknown metabolite 'h_c' created
unknown metabolite 'ru5p__L_c' created
unknown metabolite 'xu5p__D_c' created
unknown metabolite 'nadh_c' crea

Name,iJN1463
Memory address,0x01b217c31198
Number of metabolites,2154
Number of reactions,2929
Number of groups,0
Objective expression,1.0*BIOMASS_KT2440_WT3 - 1.0*BIOMASS_KT2440_WT3_reverse_d86d5
Compartments,"cytosol, extracellular space, periplasm"


It seems that P. putida does not have a D-glucose dehydrogenease. It has a Glucose 6-phosphate dehydrogenase (G6PDH2r) which might as well need to be knocked out. 

#### Add the xylose consumption pathway

In [5]:
xly1 = mock.reactions.get_by_id('XYLI1')

In [6]:
model.add_reaction(xly1)

In [7]:
xyl2 = mock.reactions.get_by_id('XYLK')
model.add_reaction(xyl2)

In [8]:
xyl3 = mock.reactions.get_by_id('XYLt2')
model.add_reaction(xyl3)

## Simulate growth on Xylose

In [9]:
r = cobra.Reaction('EX_xyl__D_e')
model.add_reaction(r)
equation = 'xyl__D_e-->'
r.build_reaction_from_string(equation, fwd_arrow='-->')
r.name = 'xyl exchange'
r.lower_bound = -1000
r.upper_bound = 0

In [10]:
glu = model.reactions.EX_glc__D_e
glu.lower_bound = 0
xyl = model.reactions.EX_xyl__D_e
xyl.lower_bound = -1000

In [11]:
model.objective.expression

1.0*BIOMASS_KT2440_WT3 - 1.0*BIOMASS_KT2440_WT3_reverse_d86d5

In [12]:
fba1 = model.optimize()

In [13]:
fba1.objective_value

2.818445334279888

In [14]:
for i in model.reactions:
    if i.flux<-100:
        print(i, i.flux)

H2Otex: h2o_e <=> h2o_p -192.0896983626632
H2Otpp: h2o_p <=> h2o_c -192.49602517117032
CO2tex: co2_e <=> co2_p -100.88364350713326
CO2tpp: co2_p <=> co2_c -100.88364350713326


In [14]:
xyl = model.reactions.EX_xyl__D_e

In [15]:
xyl.flux

-49.58049217640644

In [17]:
for i in model.reactions:
    if i.flux<-1 and "EX_" in i.id:
        print(i, i.flux)

EX_nh4_e: nh4_e <=>  -30.0
EX_o2_e: o2_e <--  -90.560118831936
EX_pi_e: pi_e <=>  -10.0
EX_so4_e: so4_e <=>  -5.861915221520185
EX_xyl__D_e: xyl__D_e <--  -49.58049217640644


In [18]:
co2 = model.reactions.EX_co2_e
co2.flux

100.88364350713326

In [19]:
glu.bounds

(0, 1000.0)

In [20]:
xyl.bounds

(-1000, 0)

In [16]:
xyl.lower_bound = 0

In [17]:
consumption = analysis_gf_sol('./pipeline/inputs/putida_ara.csv', model, mock)

EX_arab__L_e  |  arab__L_e <--   |  (-1000, 0)
EX_akg_e  |  akg_e -->   |  (0, 1000)
Old biomass (objective) bounds =  (0.0, 999999.0)


DEBUG:root:<class 'optlang.glpk_interface.Model'>
DEBUG:root:<class 'optlang.glpk_interface.Model'>


New biomass (objective) bounds =  (0.0, 1.4319489943099062)


DEBUG:root:1.0*BIOMASS_KT2440_WT3 - 1.0*BIOMASS_KT2440_WT3_reverse_d86d5
DEBUG:root:(0.0, 1.4319489943099062)
DEBUG:root:passed here
DEBUG:root:1.0*BIOMASS_KT2440_WT3 - 1.0*BIOMASS_KT2440_WT3_reverse_d86d5
cobra\util\solver.py:416 UserWarning: solver status is 'infeasible'
DEBUG:root:passed here too
DEBUG:root:-0.005056460474297036
DEBUG:root:<class 'optlang.glpk_interface.Model'>



---Carbon sources---
Starting reaction search with GapFilling . . .


...


Growth rate:  1.4319489943099062

The flux throughr EX_arab__L_e is:  -19.519630977760375

The flux throughr EX_akg_e is:  0.0

The flux throughr ARAI is:  19.519630977760375

The flux throughr RBK_L1 is:  19.519630977760375

The flux throughr RBP4E is:  19.519630977760375

---Model 2---

Reaction ARAI, solution of round 2 has been added to the model

Reaction RBK_L1, solution of round 2 has been added to the model

Reaction RBP4E, solution of round 2 has been added to the model


DEBUG:root:<class 'optlang.glpk_interface.Model'>
DEBUG:root:<class 'optlang.glpk_interface.Model'>
DEBUG:optlang.glpk_interface:Status undefined. GLPK status code returned by glp_simplex was 1



Growth rate:  1.4319489943099062

The flux throughr EX_arab__L_e is:  -19.519630977760375

The flux throughr EX_akg_e is:  0.0

The flux throughr ARAI is:  19.519630977760375

The flux throughr RBK_L1 is:  19.519630977760375

The flux throughr RBP4E is:  19.519630977760375

---Model 3---

Reaction ARABD, solution of round 3 has been added to the model

Reaction LARLC, solution of round 3 has been added to the model

Reaction LARNDH, solution of round 3 has been added to the model

Reaction L2K3dARNDH, solution of round 3 has been added to the model


DEBUG:root:<class 'optlang.glpk_interface.Model'>
DEBUG:root:<class 'optlang.glpk_interface.Model'>
DEBUG:optlang.glpk_interface:Status undefined. GLPK status code returned by glp_simplex was 1



Growth rate:  1.4319489943099062

The flux throughr EX_arab__L_e is:  -27.242023021268853

The flux throughr EX_akg_e is:  0.0

The flux throughr ARABD is:  -27.242023021268853

The flux throughr LARLC is:  27.242023021268853

The flux throughr LARNDH is:  27.242023021268853

The flux throughr L2K3dARNDH is:  27.242023021268853

---Model 4---

Reaction ARABD, solution of round 4 has been added to the model

Reaction LARLC, solution of round 4 has been added to the model

Reaction LARNDH, solution of round 4 has been added to the model

Reaction L2K3dARNDH, solution of round 4 has been added to the model

Growth rate:  1.4319489943099062

The flux throughr EX_arab__L_e is:  -27.242023021268853

The flux throughr EX_akg_e is:  0.0

The flux throughr ARABD is:  -27.242023021268853

The flux throughr LARLC is:  27.242023021268853

The flux throughr LARNDH is:  27.242023021268853

The flux throughr L2K3dARNDH is:  27.242023021268853


In [18]:
consumption

{1: ({'model': 'Model1'},
  (['ARAI', 'RBK_L1', 'RBP4E'],
   1.4319489943099062,
   {'ARAI': 19.519630977760375,
    'RBK_L1': 19.519630977760375,
    'RBP4E': 19.519630977760375},
   {'EX_arab__L_e': -19.519630977760375},
   {'EX_akg_e': 0.0})),
 3: ({'model': 'Model3'},
  (['ARABD', 'LARLC', 'LARNDH', 'L2K3dARNDH'],
   1.4319489943099062,
   {'ARABD': -27.242023021268853,
    'LARLC': 27.242023021268853,
    'LARNDH': 27.242023021268853,
    'L2K3dARNDH': 27.242023021268853},
   {'EX_arab__L_e': -27.242023021268853},
   {'EX_akg_e': 0.0}))}

In [19]:
production = dict_prod_sol('./pipeline/inputs/putida_ara_meth.csv', consumption, model, mock)

DEBUG:root:['arab__L']
DEBUG:root:<class 'optlang.glpk_interface.Model'>
DEBUG:optlang.glpk_interface:Status undefined. GLPK status code returned by glp_simplex was 1



Bounds of biomass during optimization of consumption =  (0.0, 1.4319489943099062)


DEBUG:root:<class 'optlang.glpk_interface.Model'>
DEBUG:root:passed here
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:-10.0
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__L
DEBUG:root:arab__


Bounds of biomass during optimization of production =  (0.07159744971549488, 1.4319489943099062)

The metabs to produce are:  ['akg']


DEBUG:root:passed through here
DEBUG:root:ARAI: arab__L_c <=> rbl__L_c
DEBUG:root:RBK_L1: atp_c + rbl__L_c <=> adp_c + h_c + ru5p__L_c
DEBUG:root:RBP4E: ru5p__L_c <=> xu5p__D_c
DEBUG:root:1.0*EX_akg_e - 1.0*EX_akg_e_reverse_70d85
DEBUG:root:passed here too
DEBUG:root:10.5288411349184
DEBUG:root:['ARAI', 'RBK_L1', 'RBP4E']
DEBUG:root:['ARABD', 'LARLC', 'LARNDH', 'L2K3dARNDH']



---1---
The model can already satisfy the objective


DEBUG:root:passed through here
DEBUG:root:ARABD: Lara14lac_c + h_c + nadh_c <-- arab__L_c + nad_c
DEBUG:root:LARLC: Lara14lac_c + h2o_c --> Larbn_c + h_c
DEBUG:root:LARNDH: Larbn_c --> 2dlara_c + h2o_c
DEBUG:root:L2K3dARNDH: 2dlara_c --> 25dop_c + h2o_c
DEBUG:optlang.glpk_interface:Status undefined. GLPK status code returned by glp_simplex was 1
DEBUG:root:1.0*EX_akg_e - 1.0*EX_akg_e_reverse_70d85
DEBUG:root:passed here too
DEBUG:root:10.714438716069882
DEBUG:root:['ARABD', 'LARLC', 'LARNDH', 'L2K3dARNDH']



---3---
The model can already satisfy the objective


In [20]:
production

{'1': {'1': 10.5288411349184}, '3': {'3': 10.714438716069882}}

In [21]:
final = cons_prod_dict('./pipeline/inputs/putida_ara_meth.csv', model, mock, consumption, production)

DEBUG:root:<class 'optlang.glpk_interface.Model'>
DEBUG:root:<class 'optlang.glpk_interface.Model'>
DEBUG:root:ARAI: arab__L_c <=> rbl__L_c
DEBUG:root:RBK_L1: atp_c + rbl__L_c <=> adp_c + h_c + ru5p__L_c
DEBUG:root:RBP4E: ru5p__L_c <=> xu5p__D_c
DEBUG:root:ARABD: Lara14lac_c + h_c + nadh_c <-- arab__L_c + nad_c
DEBUG:root:LARLC: Lara14lac_c + h2o_c --> Larbn_c + h_c
DEBUG:root:LARNDH: Larbn_c --> 2dlara_c + h2o_c
DEBUG:root:L2K3dARNDH: 2dlara_c --> 25dop_c + h2o_c


In [22]:
final

{'consumption_1': ({'model': 'Model1'},
  (['ARAI', 'RBK_L1', 'RBP4E'],
   1.4319489943099062,
   {'ARAI': 19.519630977760375,
    'RBK_L1': 19.519630977760375,
    'RBP4E': 19.519630977760375},
   {'EX_arab__L_e': -19.519630977760375},
   {'EX_akg_e': 0.0})),
 'production_1': {'EX_akg_e': 10.5288411349184},
 'consumption_3': ({'model': 'Model3'},
  (['ARABD', 'LARLC', 'LARNDH', 'L2K3dARNDH'],
   1.4319489943099062,
   {'ARABD': -27.242023021268853,
    'LARLC': 27.242023021268853,
    'LARNDH': 27.242023021268853,
    'L2K3dARNDH': 27.242023021268853},
   {'EX_arab__L_e': -27.242023021268853},
   {'EX_akg_e': 0.0})),
 'production_3': {'EX_akg_e': 10.714438716069882}}

The analysis with the mock model gave results:
- The two sets of reactions for isomerase and oxidative pathway has been found 

Interestingly, the oxidative pathway consist in 4 reactions instead of the 5 expected. Ketoglutarate semialdehyde dehydrogenase is not found as a solution. This might not be surprizing since the reaction is likely involved in other endogenous pathways.

1. check model
2. check KEGG


In [30]:
for i in model.reactions:
    if '25dop_c' in i.reaction:
        print(i, i.name)

1PY4h3cAH: 1py4h3c_c + h2o_c + h_c --> 25dop_c + nh4_c 1 Pyrroline 4 hydroxy 2 carboxylate aminohydrolase  decyclizing 
5DH4DGLCD: 5dh4dglc_c + h_c --> 25dop_c + co2_c + h2o_c 5 dehydro 4 deoxyglucarate dehydratase   decarboxylating 
DOPDH: 25dop_c + h2o_c + nadp_c <=> akg_c + 2.0 h_c + nadph_c Ketoglutarate semialdehyde dehydrogenase


In [27]:
akg = model.metabolites.get_by_id('akg_c')
akg.reactions

frozenset({<Reaction AATA at 0x29d402c01d0>,
           <Reaction ABTA at 0x29d402c0c50>,
           <Reaction ACOTA at 0x29d3ef974a8>,
           <Reaction AKGDH at 0x29d403bcc50>,
           <Reaction AKGDa at 0x29d403bc7f0>,
           <Reaction AKGt2rpp at 0x29d403c46d8>,
           <Reaction ALATA_L at 0x29d403e0710>,
           <Reaction APTNAT at 0x29d3f0a5be0>,
           <Reaction AROAT at 0x29d3fde2dd8>,
           <Reaction ASP3H at 0x29d3f0f3940>,
           <Reaction ASPTA at 0x29d40427dd8>,
           <Reaction DABAAT2 at 0x29d4053bc88>,
           <Reaction DAPAT at 0x29d40569710>,
           <Reaction DLYSOXGAT at 0x29d3f29c470>,
           <Reaction DOPDH at 0x29d405b2f98>,
           <Reaction GLUDxi at 0x29d3f64c978>,
           <Reaction GLUDy at 0x29d3f64c940>,
           <Reaction GLUSy at 0x29d3f657f28>,
           <Reaction GLUTARH at 0x29d40279be0>,
           <Reaction HSTPT at 0x29d3f81d518>,
           <Reaction ICDHyr at 0x29d3f845198>,
           <Reaction

In [28]:
met = model.metabolites.get_by_id('25dop_c')
met.reactions

frozenset({<Reaction 1PY4h3cAH at 0x29d3ef1f470>,
           <Reaction 5DH4DGLCD at 0x29d4029a780>,
           <Reaction DOPDH at 0x29d405b2f98>})

In [31]:
met.name

'2 5 Dioxopentanoate C5H5O4'

##### Function calls for server. 

In [ ]:
from os.path import join
import cobra
import csv
from cobra.flux_analysis import phenotype_phase_plane
from cobra.flux_analysis import gapfill

from pipeline.scripts.analysis import analysis_gf_sol, dict_prod_sol, cons_prod_dict
from pipeline.scripts.import_models import get_universal_main, get_reference_model
from pipeline.scripts.input_parser import main
from pipeline.scripts.scoring_system import (scores_evaluations, score_BB_presence, 
                            plotting_KI_BB_scores)
from pipeline.scripts.to_and_from_biobrick_wikidata import (output_for_biobrick_search, \
                                        run_harmonization_per_model)
from pipeline.scripts.call_Optknock import (run_optknock_analysis, \
                                        add_Optknock_to_analysis)
from pipeline.scripts.output_generation import (summary_output, metab_out_chain, 
                            write_data_provenance)
data_repo = "./pipeline/inputs"
model = get_reference_model(data_repo, './pipeline/inputs/putida_ara.csv')
#universal = cobra.test.create_test_model("salmonella")
universal = get_universal_main(data_repo, './pipeline/inputs/putida_ara.csv')
main('./pipeline/inputs/putida_ara.csv', universal, model)
xly1 = universal.reactions.get_by_id('XYLI1')
model.add_reaction(xly1)
xyl2 = universal.reactions.get_by_id('XYLK')
model.add_reaction(xyl2)
xyl3 = universal.reactions.get_by_id('XYLt2')
model.add_reaction(xyl3)
arai = universal.reactions.get_by_id('ARAI')
model.add_reaction(arai)
arabd = universal.reactions.get_by_id('ARABD')
model.add_reaction(arabd)
consumption = analysis_gf_sol('./pipeline/inputs/putida_ara.csv', model, universal)

In [4]:
import csv
from pipeline.scripts.input_parser import get_metabolites

In [6]:
with open('putida_ara.csv', newline='', encoding='utf-8-sig') as csvfile:
        #fieldnames = ['ï»¿metabolite_BiGG_ID', 'C-source?_y/n', 'N-source?_y/n', 'Reaction_ID', 'Reaction_name',
                      #'(Reaction_subsystem)', 'Reaction_equation', 'Reaction_lb', 'Reaction_ub',
                      #'Reaction_gene_ (ID_ and/or)', 'Metabolites', 'stoichiometry', 'metabolite_name',
                      #'Metabolite_compartment', 'Metabolite_formula', 'Metabolite_charge']
        reader = csv.DictReader(csvfile, dialect='excel')
        metabolites = get_metabolites('putida_ara.csv')
        for metab in metabolites:
            for row in reader:
                        if (metab+'_e' in row['Reaction_equation'] and
                                metab+'_p' in row['Reaction_equation']) or(
                                metab+'_e' in row['Reaction_equation'] and
                                metab+'_c' in row['Reaction_equation']) or(
                                metab+'_p' in row['Reaction_equation'] and
                                metab+'_c' in row['Reaction_equation']):
                            print(row['Reaction_ID'], row['Reaction_equation'])

ARBt2r arab__L_e + h_e --> arab__L_c + h_c
